In [1]:
import json
import pandas as pd
from datetime import datetime
import pytz
from statistics import mean, median, mode

In [3]:
#load json file
with open('shipment_data.json') as f:
    data = json.load(f)

#convert date and time
def convert_to_ist(utc_dt_str):
    if not utc_dt_str:
        return None
    utc_dt = datetime.strptime(utc_dt_str, '%Y-%m-%dT%H:%M:%S.%fZ')
    utc_dt = utc_dt.replace(tzinfo=pytz.utc)
    ist_dt = utc_dt.astimezone(pytz.timezone('Asia/Kolkata'))
    return ist_dt.strftime('%Y-%m-%d %H:%M:%S')

flattened_data = []

In [9]:
#processing shipment
for shipment in data:
    tracking_number = shipment.get('tracking_number')
    payment_type = shipment.get('payment_type')
    pickup_dt = convert_to_ist(shipment.get('pickup_datetime'))
    delivery_dt = convert_to_ist(shipment.get('delivery_datetime'))
    shipment_weight = shipment.get('shipment_weight')

    pickup_pincode = shipment.get('pickup', {}).get('pincode')
    pickup_city = shipment.get('pickup', {}).get('city')
    pickup_state = shipment.get('pickup', {}).get('state')

    drop_pincode = shipment.get('drop', {}).get('pincode')
    drop_city = shipment.get('drop', {}).get('city')
    drop_state = shipment.get('drop', {}).get('state')

    out_for_delivery_events = shipment.get('out_for_delivery_events', [])
    out_for_delivery_dates = [convert_to_ist(event) for event in out_for_delivery_events]

    if pickup_dt and delivery_dt:
        pickup_dt_obj = datetime.strptime(pickup_dt, '%Y-%m-%d %H:%M:%S')
        delivery_dt_obj = datetime.strptime(delivery_dt, '%Y-%m-%d %H:%M:%S')
        days_taken = (delivery_dt_obj - pickup_dt_obj).days
    else:
        days_taken = None

    delivery_attempts = len(out_for_delivery_dates)
    same_day_delivery = False

    if delivery_dt:
        delivery_day = delivery_dt.split(' ')[0]
        if delivery_day in [dt.split(' ')[0] for dt in out_for_delivery_dates]:
            same_day_delivery = True

    if delivery_dt:
        if same_day_delivery:
            delivery_attempts = 1
        else:
            delivery_attempts += 1
            
    #flattened record
    flattened_data.append({
        'Tracking number': tracking_number,
        'Payment type': payment_type,
        'Pickup Date Time in IST': pickup_dt,
        'Delivery Date Time in IST': delivery_dt,
        'Days taken for delivery': days_taken,
        'Shipment weight': shipment_weight,
        'Pickup Pincode': pickup_pincode,
        'Pickup City': pickup_city,
        'Pickup State': pickup_state,
        'Drop Pincode': drop_pincode,
        'Drop City': drop_city,
        'Drop State': drop_state,
        'Number of delivery attempts needed': delivery_attempts
    })
    
shipment_df = pd.DataFrame(flattened_data)

# Export flattened shipment data to CSV
shipment_df.to_csv('F:\\Shipment_Swift_\\flattened_shipments.csv', index=False)

# Prepare valid lists for summary
days_list = [d for d in shipment_df['Days taken for delivery'] if d is not None]
attempts_list = [d for d in shipment_df['Number of delivery attempts needed'] if d is not None]

summary = {}

if days_list:
    summary['Mean days taken for delivery'] = mean(days_list)
    summary['Median days taken for delivery'] = median(days_list)
    summary['Mode days taken for delivery'] = mode(days_list)
else:
    summary['Mean days taken for delivery'] = None
    summary['Median days taken for delivery'] = None
    summary['Mode days taken for delivery'] = None

if attempts_list:
    summary['Mean delivery attempts'] = mean(attempts_list)
    summary['Median delivery attempts'] = median(attempts_list)
    summary['Mode delivery attempts'] = mode(attempts_list)
else:
    summary['Mean delivery attempts'] = None
    summary['Median delivery attempts'] = None
    summary['Mode delivery attempts'] = None

# Convert summary to DataFrame
summary_df = pd.DataFrame([summary])

# Export summary to CSV
summary_df.to_csv('F:\\Shipment_Swift_\\summary_statistics.csv', index=False)

# Display output
shipment_df.head(), summary_df

(  Tracking number Payment type Pickup Date Time in IST  \
 0            None         None                    None   
 1            None         None                    None   
 2            None         None                    None   
 3            None         None                    None   
 4            None         None                    None   
 
   Delivery Date Time in IST Days taken for delivery Shipment weight  \
 0                      None                    None            None   
 1                      None                    None            None   
 2                      None                    None            None   
 3                      None                    None            None   
 4                      None                    None            None   
 
   Pickup Pincode Pickup City Pickup State Drop Pincode Drop City Drop State  \
 0           None        None         None         None      None       None   
 1           None        None         None        